## Funciones necesarias para el funcionamiento del proyecto

Datos de prueba (genéricos)

In [1]:
#Datos prueba

lat = 38.1609
lon = -0.801499
orient = 10
incl = 25
ppico = 4.62 #kW pico
fecha = "2021-03-23"
fecha_a_usar = fecha

Excentricidad de la tierra

In [2]:
import math 

def excentricidad(dia):
    e_0 = 1 + 0.033 * math.cos((2*math.pi*dia)/365)
    return e_0

Declinacion de la tierra

In [3]:
import math 

def gamma_declinacion(dia):
    gamma = 2.0 * math.pi *(dia-1)/365.0
    return gamma

In [4]:
import math 

def angulo_declinacion_solar(dia):
    gamma = gamma_declinacion(dia)
    delta = 0.006918 - 0.399912 * math.cos(gamma) + 0.070257 * math.sin(gamma) - 0.006758 * math.cos(2*gamma) + 0.000907 * math.sin(2* gamma) - 0.002697 * math.cos(3*gamma) + 0.001480 * math.sin(3*gamma)
    return delta #(radianes)

Ángulo de salida del sol

In [5]:
import math 

def angulo_salida_sol(latitud, declinacion):
    w = math.acos( -math.tan(latitud) * math.tan(declinacion))
    return w

Horas hasta el mediodia solar

In [6]:
import math 

def horas_mediodia(wsr):
    horas = wsr * 12/math.pi
    return horas

def segundos_(horas_mediodia):
    segundos = (horas_mediodia - math.trunc(horas_mediodia)) * 3600
    return segundos


def hora_inicial_(horas_mediodia):
    hora_inicial = 12 - math.trunc(horas_mediodia)
    return hora_inicial


def hora_final_(horas_mediodia):
    hora_final = 12 + math.trunc(horas_mediodia)
    return hora_final


Ángulo solar horario

In [7]:
import math 

def angulo_solar_primera_ultima_hora(segundos, w_sr):
    w = w_sr - ((segundos/3600)/2 * math.pi/12)
    return w


def angulo_solar(hora):
    w = (hora - 12 - 0.5) * math.pi/12
    return w

Altura solar horaria

In [8]:
import math 

def altura_solar(declinacion, latitud, angulo_solar_hora):
    altura = math.asin(math.sin(declinacion) * math.sin(latitud) + math.cos(declinacion) * math.cos(latitud) * math.cos(angulo_solar_hora))
    return altura

Acimut solar horario

In [9]:
import math 

def acimut_solar(altura_solar, latitud, declinacion, angulo_horario):
    if(latitud > 0):
        acimut = math.acos((math.sin(altura_solar) * math.sin(latitud) - math.sin(declinacion))/(math.cos(altura_solar) * math.cos(latitud)))
    else:
        acimut = math.acos((math.sin(declinacion) * math.sin(latitud) - math.cos(declinacion) * math.sin(latitud) * math.cos(angulo_horario))/math.cos(declinacion))
    return acimut

Radiación extraterrestre incidente

In [10]:
import math 

K_solar = 1367 #W/m^2

def radiacion_incidente_extraterrestre(excentricidad, altura_solar_hora):
    r_incidente = K_solar * excentricidad * math.sin(altura_solar_hora)
    return r_incidente


def radiacion_incidente_extraterrestre_primera_ultima_hora(excentricidad, altura_solar_hora, segundos):
    radiacion = radiacion_incidente_extraterrestre(excentricidad, altura_solar_hora)
    radiacion = radiacion * segundos / 3600
    return radiacion

Obtener dataframe de medidas horarias para una inclinación, latitud, longitud, altura y acimut (orientación)

In [11]:
import math 
import pandas as pd
import itertools



def df_energia_solar(lat, lon, orientacion = 0, inclinacion = 0):
    lat = lat * math.pi/180
    lon = lon * math.pi/180
    inclinacion = inclinacion * math.pi/180
    orientacion = orientacion * math.pi/180
  
    mes = [1]*31 + [2]*28 + [3]*31 + [4]*30 + [5]*31 + [6]*30 + [7]*31 + [8]*31 + [9]*30 + [10]*31 + [11]*30 + [12]*31
    dia = list(range(1,32)) + list(range(1,29)) + list(range(1,32)) + list(range(1,31)) + list(range(1,32)) + list(range(1,31)) + list(range(1,32)) + list(range(1,32)) + list(range(1,31)) + list(range(1,32)) + list(range(1,31)) + list(range(1,32))
    dia_juliano = list(range(1,366))
  
    df = pd.DataFrame({'mes' : mes,
                       'dia' : dia,
                       'dia_juliano' : dia_juliano})
    df.insert(3, "declinacion_solar", list(map(angulo_declinacion_solar, dia_juliano)), True) 
    df.insert(4, "excentricidad_diaria", list(map(excentricidad, dia_juliano)), True) 
    df.insert(5, "w_sr", list(map(angulo_salida_sol, df["declinacion_solar"], itertools.repeat(lat, len(df["declinacion_solar"])))), True) 
    df.insert(6, "horas_mediodia", list(map(horas_mediodia,df["w_sr"])), True) 
    df.insert(7, "duracion_dia", df["horas_mediodia"]*2, True) 
    df.insert(8, "hora_inicial", list(map(hora_inicial_, df["horas_mediodia"])), True) 
    df.insert(9, "hora_final", list(map(hora_final_, df["horas_mediodia"])), True) 
    df.insert(10, "segundos", list(map(segundos_, df["horas_mediodia"])), True) 
    
    
    df_radiacion = pd.DataFrame(dia_juliano[0:365])
    df_radiacion.columns = ['dia_juliano']
    df_altura = pd.DataFrame(dia_juliano[0:365])
    df_altura.columns = ['dia_juliano']
    df_acimut = pd.DataFrame(dia_juliano[0:365])
    df_acimut.columns = ['dia_juliano']
    df_angulo_solar = pd.DataFrame(dia_juliano[0:365])
    df_angulo_solar.columns = ['dia_juliano']
    
    # Para cada hora del día, se obtinene las diferenets variable solares en función del día del año
    for h in range(1,25):
        w = []
        altura = []
        acimut = []
        radiacion = []
    
        for d in range(0,365):
            hora_inicial = df["hora_inicial"][d]
            hora_final = df["hora_final"][d]
            segundos = df["segundos"][d]
            w_sr = df["w_sr"][d]
            delta = df["declinacion_solar"][d] 
            E = df["excentricidad_diaria"][d]          
                   
            
            if(h == (hora_inicial) or h == (hora_final+1)):
                w_d = angulo_solar_primera_ultima_hora(segundos, w_sr)
                altura_d = altura_solar(delta, lat, w_d)
                acimut_d = acimut_solar(altura_d, lat, delta, w_d)       
                if(h == (hora_inicial)):
                    acimut_d = -acimut_d
                    w_d = -w_d              
                rad_d = radiacion_incidente_extraterrestre_primera_ultima_hora(E, altura_d, segundos)
                
            elif(h >= (hora_inicial+1) and h < (hora_final+1)):
                w_d = angulo_solar(h)
                altura_d = altura_solar(delta, lat, w_d)
                acimut_d = acimut_solar(altura_d, lat, delta, w_d)
                if(h < 13):
                    acimut_d = -acimut_d
                rad_d = radiacion_incidente_extraterrestre(E, altura_d)
                
            else:
                w_d = 0
                altura_d = 0
                acimut_d = 0
                rad_d = 0
        

            w.append(w_d)
            altura.append(altura_d)
            acimut.append(acimut_d)
            radiacion.append(rad_d)
            
        # angulo solar
        df_angulo_solar.insert(len(df_angulo_solar.columns), str("W_{}_{}".format(h-1, h)), w)
        
        # altura solar
        df_altura.insert(len(df_altura.columns), str("Altura_{}_{}".format(h-1, h)), altura)

        # acimut solar
        df_acimut.insert(len(df_acimut.columns), str("Acimut_{}_{}".format(h-1, h)), acimut)

        # Radiacion extraterrestre
        df_radiacion.insert(len(df_radiacion.columns), str("Gh0_{}_{}".format(h-1, h)), radiacion)

    df = df.merge(df_angulo_solar, left_on = "dia_juliano", right_on = "dia_juliano")
    df = df.merge(df_altura, left_on="dia_juliano", right_on="dia_juliano")
    df = df.merge(df_acimut, left_on="dia_juliano", right_on="dia_juliano")
    df = df.merge(df_radiacion, left_on="dia_juliano", right_on="dia_juliano")

    
    return df

Radiación global

In [12]:
def radiacion_global(r_directa, r_difusa, r_reflejada):
    r_global = r_directa + r_difusa + r_reflejada
    return r_global

Índice de transparencia atmosférico

In [13]:
def indice_transparencia(r_global, r_extraterrestre):
    i_transparencia = r_global/r_extraterrestre
    return i_transparencia

Radiación difusa

In [14]:
import math 

def radiacion_difusa_horizontal(indice_transparencia, r_global):
    if(indice_transparencia >= 0 and indice_transparencia <= 0.22):
        r_difusa = r_global *(1 - 0.09 * indice_transparencia)
    elif(0.22 < indice_transparencia and indice_transparencia <= 0.8):
        r_difusa = r_global * (0.9511 - 0.16 * indice_transparencia + 4.388 * indice_transparencia**2 - 16.638 * indice_transparencia**3 + 12.336 * indice_transparencia**4)
    else:
        r_difusa = r_global * 0.165
    
    return r_difusa

def radiacion_difusa_inclinada(r_difusa, beta):
    if(beta != 0):
        r_difusa_beta = (1 + math.cos(beta))/2 * r_difusa
        
    return r_difusa_beta

Radiación reflejada

In [15]:
import math 

def radiacion_reflejada(radiacion_global, albedo = 0.2, beta = 0):
    radiacion_reflejada = radiacion_global * albedo * (1- math.cos(beta))/2
    
    return radiacion_reflejada

Radiación directa

In [16]:
import math 

def wsrm_(orientacion, beta, latitud, declinacion, angulo_salida_solar):
    
    if(orientacion < 0):
        signo = -1
    else:
        signo = 1
    
    A = math.sin(latitud)/math.tan(orientacion) + math.cos(latitud)/(math.sin(orientacion) * math.tan(beta))
    B = -math.tan(declinacion)*(math.sin(latitud)/(math.sin(orientacion)*math.tan(beta)) - math.cos(latitud)/math.tan(orientacion))
  
    wsrm = math.acos((A*B + signo*math.sqrt(A*A-B*B+1))/(A*A+1))
    wsrm = - min(wsrm,angulo_salida_solar) 
  
    return wsrm


def wsrt_(orientacion, beta, latitud, declinacion, angulo_salida_solar):
    
    if (orientacion < 0):
        signo = -1
    else:
        signo <- 1
  
    A = math.sin(latitud)/math.tan(orientacion) + math.cos(latitud)/(math.sin(orientacion) * math.tan(beta))
    B = -math.tan(declinacion)*(math.sin(latitud)/(math.sin(orientacion)*math.tan(beta)) - math.cos(latitud)/math.tan(orientacion))
  
    wsrt = math.acos((A*B - signo*math.sqrt(A*A-B*B+1))/(A*A+1))
    wsrt = min(wsrt, angulo_salida_solar)
    
    return wsrt



def radiacion_directa(r_global, r_difusa, acimut_solar_h, altura_solar_h, declinacion, latitud, angulo_solar_h, angulo_salida_solar, beta = 0, orientacion = 0):
    
    r_directa = r_global - r_difusa
  
    if (beta != 0):
        
        if(orientacion <0 and acimut_solar_h > 0):
            wsrt = wsrt_(orientacion, beta, latitud, declinacion, angulo_salida_solar)
            if(angulo_solar_h > wsrt):
                angulo_solar_h = wsrt

        if(orientacion >0 and acimut_solar_h <0):
            wsrm = wsrm_(orientacion, beta, latitud, declinacion, angulo_salida_solar) 
            if(angulo_solar_h < wsrm):
                angulo_solar_h = wsrm

        costhita = math.sin(declinacion) * math.sin(latitud) * math.cos(beta) - math.sin(declinacion) * math.cos(latitud) * math.sin(beta) * math.cos(orientacion) + math.cos(declinacion) * math.cos(latitud) * math.cos(beta) * math.cos(angulo_solar_h) + math.cos(declinacion) * math.sin(latitud) * math.sin(beta) * math.cos(orientacion) * math.cos(angulo_solar_h) + math.cos(declinacion) * math.sin(beta) * math.sin(orientacion) * math.sin(angulo_solar_h)

        if(costhita < 0):
            costhita = 0

        costhitaz = math.sin(declinacion) * math.sin(latitud) + math.cos(declinacion) * math.cos(latitud) * math.cos(angulo_solar_h)

        
        if(costhitaz > 0):
            rb = costhita/costhitaz
        else:
            rb = 1

        r_directa = r_directa * rb


    return r_directa

Obtención de la potencia generada por los paneles fotovoltaicos

In [17]:
import math 
import numpy as np

def temperatura_instalacion(temperatura_ambiente, irradiacion_solar, velocidad_viento, m = -3.56, n = -0.079):
    temperatura = []
    for i in range(0, len(temperatura_ambiente)):
        temperatura.append(temperatura_ambiente[i] + irradiacion_solar[i] * math.exp(m + n * velocidad_viento[i]))
    return temperatura


def potencia_generada(t_ambiente, irr_solar, v_viento, ppico):
    
    g_ref = 1000 #W/m^2
    coef_y = -0.0048
    t_ref = 25
    P_ref = 1000 * ppico
  
    coefCC = 0.97 * 0.98 * 0.98 * 0.98
    efinversor = 0.95
    perdCA = 0.99
    T_m = temperatura_instalacion(t_ambiente, irr_solar, v_viento)
  
    potencia = P_ref * np.array(irr_solar)/g_ref * (1 + coef_y * (np.array(T_m) - t_ref))
    potencia = potencia * coefCC * efinversor * perdCA
  
    return potencia

Función auxiliar para calcular la radiación global horaria necesaria para obtener la producción fotovoltaica horaria

In [18]:
def calculo_radiacion_horaria(r_global, K, acimut_sol, altura_sol, angulo_sol, decl, latitud, inclinacion, orientacion, angulo_salida_solar):
    
    r_difusa_horiz = []
    r_difusa_incl = []
    r_directa_h = []
    r_reflejada_h = []
    r_global_h = []
    
    for h in range(0,len(r_global)):
        k = K[h]
        rg = r_global[h]
        acimut_h = acimut_sol[h]
        altura_h = altura_sol[h]
        angulo_h = angulo_sol[h]
        declinacion = decl

        r_difusa  = radiacion_difusa_horizontal(k, rg)
        r_difusa_horiz.append(r_difusa)
    
        r_directa = radiacion_directa(r_global = rg, 
                                       r_difusa = r_difusa, 
                                       beta = inclinacion, 
                                       orientacion = orientacion, 
                                       acimut_solar_h = acimut_h, 
                                       altura_solar_h = altura_h, 
                                       declinacion = declinacion, 
                                       latitud = latitud, 
                                       angulo_solar_h = angulo_h,
                                       angulo_salida_solar = angulo_salida_solar)
        r_directa_h.append(r_directa)
    
        r_difusa_beta = radiacion_difusa_inclinada(r_difusa = r_difusa, beta = inclinacion)
        r_difusa_incl.append(r_difusa_beta)

        r_reflejada = radiacion_reflejada(radiacion_global = rg, beta = inclinacion)
        r_reflejada_h.append(r_reflejada)

        r_global_plano = radiacion_global(r_directa = r_directa, r_difusa = r_difusa_beta, r_reflejada = r_reflejada)
        r_global_h.append(r_global_plano)

    df_radiaciones = pd.DataFrame({'r_difusa_incl':r_difusa_incl, 'r_reflejada_h':r_reflejada_h, 'r_directa_h':r_directa_h, 'r_global_h':r_global_h})
    return df_radiaciones


Función que calcula la distancia euclidea o de Manhattan entre dos puntos

In [19]:
import math 

def distancia(lat1, lon1, lat2, lon2, distancia = "euclidea"):
    
    if(distancia == "euclidea"):
        dist = math.sqrt((lat1 - lat2)**2 + (lon1 -lon2)**2)
    
    elif(distancia == "manhattan"):
        dist = abs(lat1 - lat2) + abs(lon1 -lon2)
  
    return dist 

Saber si un día tiene horario invierno o verano en España

In [20]:
from datetime import date, datetime

def verano_invierno(now):
    
    Y = 2000 #año bisiesto genérico
    inv_ver = [(1, (date(Y,  1,  1),  date(Y,  3, 26))),
               (2, (date(Y,  3, 27),  date(Y,  10, 30))),
               (1, (date(Y, 10, 31),  date(Y, 12, 31)))]
    # 1 Invierno, 2 Verano
    
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in inv_ver
                if start <= now <= end)

Función de cálculo

In [21]:
from datetime import timezone, datetime, date, timedelta
import math

def calcularEnergia(lat, lon, orient, incl, ppico, fecha, dia_Gh, ta):
       
    fecha = str(fecha)
    
    # Dia y mes actual
  
    actualDay = datetime.strptime(fecha, '%Y-%m-%d').day
    actualMonth = datetime.strptime(fecha, '%Y-%m-%d').month
 
    
    # Se calcula el dataframe con la informacion de radiacion (excentricidad, wsr...) para la lat, lon, orient e inclinacion que se esta procesando
    
    df_solar_energy = df_energia_solar(lat, lon, orient, incl) 
    
    # Nos quedamos con los datos de radiacion para el dia
    
    calculos_dia = df_solar_energy[df_solar_energy["mes"] == actualMonth]
    calculos_dia = calculos_dia[calculos_dia["dia"] == actualDay]
   
    acimut_h_dia = []
    altura_h_dia = []
    angulo_h_dia = []
    gh0_h_dia = []
    
    [acimut_h_dia.append("Acimut_" + str(i) + "_" + str(i+1)) for i in range(4,20)]
    [altura_h_dia.append("Altura_" + str(i) + "_" + str(i+1)) for i in range(4,20)]
    [angulo_h_dia.append("W_" + str(i) + "_" + str(i+1)) for i in range(4,20)]
    [gh0_h_dia.append("Gh0_" + str(i) + "_" + str(i+1)) for i in range(4,20)]

    
    dia_Acimut_h = calculos_dia[acimut_h_dia]     #Valores horarios de acimut para el dia
    dia_Altura_h = calculos_dia[altura_h_dia]     #Valores horarios de altura solar para el dia
    dia_Angulo_h = calculos_dia[angulo_h_dia]     #Valores horarios de angulo solar para el dia
    dia_declinacion = calculos_dia["declinacion_solar"]   #Valores horarios de declinacion para el dia
    dia_Gh0 = calculos_dia[gh0_h_dia]   #Valores horarios de Gh0 (Radiacion extraterrestre) para el dia
  
    dia_Kh  = indice_transparencia(r_global = dia_Gh, r_extraterrestre = dia_Gh0)  #Valores horarios de indice de transparencia para el dia
    dia_Kh_lista = list(pd.DataFrame(dia_Kh).reset_index(drop=True).iloc[0].isnull())
    for i in range(0, len(dia_Kh_lista)):
        if (dia_Kh_lista[i] == True):
            dia_Kh[dia_Kh.columns[i]] = 0
        

    inclinacion = incl * math.pi/180
    orientacion = orient * math.pi/180
    lat = lat * math.pi/180
    angulo_salida_solar = angulo_salida_sol(lat, dia_declinacion)
    
    
    # Obtener dataframe de radiacion para el dia actual 
  
    df_radiacion_horaria = calculo_radiacion_horaria(dia_Gh,
                                                    list(dia_Kh.iloc[0]), 
                                                    list(dia_Acimut_h.iloc[0]), 
                                                    list(dia_Altura_h.iloc[0]), 
                                                    list(dia_Angulo_h.iloc[0]),
                                                    dia_declinacion,
                                                    lat,
                                                    inclinacion,
                                                    orientacion,
                                                    angulo_salida_solar)
     
    # Temperatura ambiente y velocidad de viento
    
    temperatura_ambiente = ta
    
    vel_viento = []
    for i in range (0, len(temperatura_ambiente)):
        vel_viento.append(3.5)
    
    # Se calcula la energia
    
    potencia_dia_h = potencia_generada(temperatura_ambiente, list(df_radiacion_horaria["r_global_h"]), vel_viento, ppico)
        
        
    # Saber cuantas horas adelantar los datos para pasarlos a hora local
    
    horas_atrasar = verano_invierno(datetime.strptime(fecha, '%Y-%m-%d').date())
    
    if (horas_atrasar == 1):
        potencia_dia_h = [0,0,0,0,0] + list(potencia_dia_h) + [0,0,0]
    else:
        potencia_dia_h = [0,0,0,0,0,0] + list(potencia_dia_h) + [0,0]
        
        
    return potencia_dia_h


Se convierte a .py para poder utilizarlo en ``Interfaz.ipynb``

In [22]:
#! jupyter nbconvert --to script 'Funciones_solares.ipynb'